# Task 4
This serves as a template which will guide you through the implementation of this task. It is advised to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

First, we import necessary libraries:

In [11]:
import numpy as np
import pandas as pd
import torch
from datasets import Dataset, DatasetDict
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments,
)
# Add any other imports you need here

Depending on your approach, you might need to adapt the structure of this template or parts not marked by TODOs.
It is not necessary to completely follow this template. Feel free to add more code and delete any parts that are not required.

In [12]:
def clear_cache():
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()

In [13]:
# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 4  # TODO: Set the batch size according to both training performance and available memory
NUM_EPOCHS = 10  # TODO: Set the number of epochs

# Load test data
test_val = pd.read_csv("test_no_score.csv")
test_val["text"] = test_val["title"] + " " + test_val["sentence"]
test_val = test_val[["text"]]


# Load train data
train_val = pd.read_csv("train.csv")

train_val["text"] = train_val["title"] + " " + train_val["sentence"]
train_val["label"] = train_val["score"]
train_val = train_val[["text", "label"]]

# Load datasets
train_dataset = Dataset.from_pandas(train_val)
test_dataset = Dataset.from_pandas(test_val)

train_test_split = train_dataset.train_test_split(test_size=0.1)
dataset = DatasetDict(
    {"train": train_test_split["train"], "validation": train_test_split["test"]}
)


print(dataset["train"][0])

{'text': "Why I Bought This CD Forgotten Dreams by David Pietro was such a strong CD that I've ordered this latest CD without reading any reviews. I know I'll love it!", 'label': 9.377984268162235}


In [14]:
# Tokenize
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def tokenizer_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenizer_function, batched=True)
print(tokenized_dataset["train"][0])

model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=1)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    logging_steps=10,
    logging_first_step=True,
    use_mps_device=True,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.squeeze()
    mse = mean_squared_error(labels, preds)
    return {"mse": mse}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [ ]:
trainer.train()

model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

/Users/arturvonruffer/anaconda3/envs/iml/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


AssertionError: EarlyStoppingCallback requires load_best_model_at_end = True

In [ ]:
# Do predictions
tokenized_test_dataset = test_dataset.map(tokenizer_function, batch=True)

predictions = trainer.predict(tokenized_test_dataset)
pred_scores = predictions.predictions.squeeze()

with open("results.txt", "w") as f:
    for val in pred_scores:
        f.write(f"{val}\n")

print("Saved results")

In [ ]:
# model.eval()
# with torch.no_grad():
#     results = []
#     for batch in tqdm(test_loader, total=len(test_loader)):
#         batch = batch.to(DEVICE)

#         # TODO: Set up evaluation loop

#     with open("result.txt", "w") as f:
#         for val in np.concatenate(results):
#             f.write(f"{val}\n")

NameError: name 'torch' is not defined